In [1]:
import sqlite3

import os
os.environ["JAVA_HOME"] = "/mnt/extproj/projekte/textmining/jdk/openlogic-openjdk-11.0.22+7-linux-x64"

from collections import defaultdict
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import Window

from collections import Counter


In [2]:
dbFile = '/mnt/extproj/projekte/textmining/mx_frontend/cons_evidences.db'

In [3]:
os.remove(dbFile)

In [4]:
con =  sqlite3.connect(dbFile)
cur = con.cursor()

In [5]:

cur.execute("DROP TABLE IF EXISTS mx;")
cur.execute("DROP TABLE IF EXISTS mx_int;")
cur.execute("DROP TABLE IF EXISTS mx_annot;")
cur.execute("DROP TABLE IF EXISTS mx_sent;")
cur.execute("DROP TABLE IF EXISTS mx_dates;")


ct = """CREATE TABLE "mx" (
"index" INTEGER PRIMARY KEY AUTOINCREMENT,
"miRNA_family" TEXT,
"gene_family" TEXT,
"interaction" TEXT,
"organisms" TEXT,
"evidence_documents" TEXT,
"sent_evidences" TEXT,
"evidence_count" INTEGER,
"sent_count" INTEGER,
"is_consensus" INTEGER
)"""
cur.execute(ct)

ct = """CREATE TABLE "mx_int" (
"index" INTEGER PRIMARY KEY AUTOINCREMENT,
"miRNA_family" TEXT,
"gene_family" TEXT,
"interaction" TEXT,
"evidence_documents" TEXT
)"""
cur.execute(ct)

ct = """CREATE TABLE "mx_annot" (
"index" INTEGER PRIMARY KEY AUTOINCREMENT,
"doc_id" TEXT,
"annotation" TEXT,
"concept_id" TEXT,
"concept" TEXT,
"num_occurrences" INTEGER,
"sent_evidences" TEXT
)"""
cur.execute(ct)

ct = """CREATE TABLE "mx_sent" (
"index" INTEGER PRIMARY KEY AUTOINCREMENT,
"doc_id" TEXT,
"sent_id" TEXT,
"sentence" TEXT
)"""
cur.execute(ct)

ct = """CREATE TABLE "mx_dates" (
"index" INTEGER PRIMARY KEY AUTOINCREMENT,
"doc_id" TEXT,
"date" TEXT
)"""
cur.execute(ct)


cur.execute("CREATE INDEX IF NOT EXISTS mx_mirna on mx (miRNA_family);")
cur.execute("CREATE INDEX IF NOT EXISTS mx_gene on mx (gene_family);")
cur.execute("CREATE INDEX IF NOT EXISTS mx_interaction on mx (interaction);")
cur.execute("CREATE INDEX IF NOT EXISTS mx_evs on mx (evidence_documents);")

cur.execute("CREATE INDEX IF NOT EXISTS mx_int_mirna on mx_int (miRNA_family);")
cur.execute("CREATE INDEX IF NOT EXISTS mx_int_gene on mx_int (gene_family);")
cur.execute("CREATE INDEX IF NOT EXISTS mx_int_interaction on mx_int (interaction);")
cur.execute("CREATE INDEX IF NOT EXISTS mx_int_evs on mx_int (evidence_documents);")

cur.execute("CREATE INDEX IF NOT EXISTS mx_annot_doc on mx_annot (doc_id);")
cur.execute("CREATE INDEX IF NOT EXISTS mx_annot_concept on mx_annot (concept);")
cur.execute("CREATE INDEX IF NOT EXISTS mx_annot_annotation on mx_annot (annotation);")
cur.execute("CREATE INDEX IF NOT EXISTS mx_annot_occs on mx_annot (num_occurrences);")

cur.execute("CREATE INDEX IF NOT EXISTS mx_sent_doc on mx_sent (doc_id);")
cur.execute("CREATE INDEX IF NOT EXISTS mx_sent_sentid on mx_sent (sent_id);")

cur.execute("CREATE INDEX IF NOT EXISTS mx_dates_doc on mx_dates (doc_id);")

In [6]:
spark = None

In [7]:
if not "spark" in globals() or spark is None:
    print("Creating builder")
    spark = SparkSession.builder.\
            config("spark.executor.memory", "70g").\
            config("spark.driver.memory", "50g").\
            config("spark.memory.offHeap.enabled",True).\
            config("spark.memory.offHeap.size","16g").\
            config("spark.driver.maxResultSize", "4g").\
            config("spark.sql.shuffle.partitions", 300).\
            appName('mirexplore').getOrCreate()

Creating builder


/mnt/extproj/projekte/bartelt/software/miniconda3/envs/mirexplore/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [8]:
schema_int = StringType()

interaction_udf = udf(
    lambda x: ";".join([y for y in list(x)]),
    schema_int
)

def transform_sent(x):
    sents = []
    for y in list(x):
        docid = y[0]
        loc1 = y[1]
        loc2 = y[2]
        
        sents.append( str( (docid, tuple(loc1), tuple(loc2)) ) )
    
    return ";".join(sents)


sent_udf = udf(
    lambda x: transform_sent(x),
    schema_int
)

In [9]:
df_int = spark.read.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_mirna_gene_consensus_parquet")
df_int.show(n=5)

+------------+-----------+----------------+---------+--------------------+--------------------+--------------+----------+------------+
|miRNA_family|gene_family|     interaction|organisms|  evidence_documents|      sent_evidences|evidence_count|sent_count|is_consensus|
+------------+-----------+----------------+---------+--------------------+--------------------+--------------+----------+------------+
|    miR-8485|       INSR| {MIR_GENE, NEU}|  hsa;mmu|        [PMC5979287]|[{PMC5979287.3.54...|             1|         1|           1|
|    miR-8485|       IRS2| {MIR_GENE, NEU}|  hsa;mmu|        [PMC5979287]|[{PMC5979287.3.54...|             1|         1|           1|
|    miR-8485|      ITGAM| {GENE_MIR, NEU}|  hsa;mmu|        [PMC9639483]|[{PMC9639483.3.22...|             1|         1|           1|
|    miR-8485|    LAMTOR3| {GENE_MIR, NEU}|  hsa;mmu|        [PMC9358144]|[{PMC9358144.3.15...|             1|         1|           0|
|    miR-8485|    LAMTOR3|{MIR_GENE, DOWN}|  hsa;mmu|[3

In [10]:
df_intdoc = df_int.select(["miRNA_family", "gene_family", "interaction", "evidence_documents"]).withColumn("interaction", interaction_udf("interaction")).withColumn("evidence_documents", explode("evidence_documents"))
df_intdoc.show(n=5)

+------------+-----------+-------------+------------------+
|miRNA_family|gene_family|  interaction|evidence_documents|
+------------+-----------+-------------+------------------+
|    miR-8485|       INSR| MIR_GENE;NEU|        PMC5979287|
|    miR-8485|       IRS2| MIR_GENE;NEU|        PMC5979287|
|    miR-8485|      ITGAM| GENE_MIR;NEU|        PMC9639483|
|    miR-8485|    LAMTOR3| GENE_MIR;NEU|        PMC9358144|
|    miR-8485|    LAMTOR3|MIR_GENE;DOWN|          35959437|
+------------+-----------+-------------+------------------+
only showing top 5 rows



In [11]:
df_int = spark.read.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_mirna_gene_consensus_parquet")

df_cons2 = df_int.withColumn("interaction", interaction_udf("interaction"))\
                .withColumn("evidence_documents", concat_ws(";",col("evidence_documents")))\
                .withColumn("sent_evidences", sent_udf("sent_evidences"))
                
                #.withColumn("organisms", concat_ws(";",col("organisms"))) \

In [12]:
df_cons2.show(n=5)

+------------+-----------+-------------+---------+-------------------+--------------------+--------------+----------+------------+
|miRNA_family|gene_family|  interaction|organisms| evidence_documents|      sent_evidences|evidence_count|sent_count|is_consensus|
+------------+-----------+-------------+---------+-------------------+--------------------+--------------+----------+------------+
|    miR-8485|       INSR| MIR_GENE;NEU|  hsa;mmu|         PMC5979287|('PMC5979287.3.54...|             1|         1|           1|
|    miR-8485|       IRS2| MIR_GENE;NEU|  hsa;mmu|         PMC5979287|('PMC5979287.3.54...|             1|         1|           1|
|    miR-8485|      ITGAM| GENE_MIR;NEU|  hsa;mmu|         PMC9639483|('PMC9639483.3.22...|             1|         1|           1|
|    miR-8485|    LAMTOR3| GENE_MIR;NEU|  hsa;mmu|         PMC9358144|('PMC9358144.3.15...|             1|         1|           0|
|    miR-8485|    LAMTOR3|MIR_GENE;DOWN|  hsa;mmu|35959437;PMC9358144|('PMC9358144.

In [13]:
df_cons2.select(col("organisms")).distinct().show()

+---------+
|organisms|
+---------+
|  hsa;mmu|
|      mmu|
|      hsa|
+---------+



In [14]:
df_sent = spark.read.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_document_sentences")
df_sent.show(n=5)

+-----------+-----------------+--------------------+
|     doc_id|          sent_id|            sentence|
+-----------+-----------------+--------------------+
|PMC10208224|PMC10208224.3.109|(B) Penalty plot ...|
|PMC10208224|PMC10208224.3.110|(C) Distribution ...|
|PMC10208224|PMC10208224.3.111|(D) The error var...|
|PMC10208224|PMC10208224.3.112|(E) Identificatio...|
|PMC10208224|PMC10208224.3.113|(F) 7 immune cell...|
+-----------+-----------------+--------------------+
only showing top 5 rows



In [15]:
def transform_sent2(x):
    sents = []

    for y in list(x):
        
        sentid = y["sent"]
        loc = tuple(y["loc"])
        
        sents.append( "{},{},{}".format(sentid, loc[0], loc[1]) )
    
    return "|".join(sents)


sent_udf2 = udf(
    lambda x: transform_sent2(x),
    schema_int
)

In [16]:
df_annot = spark.read.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_document_annotations")
df_annot2 = df_annot.withColumn("num_occurrences", size("sent_evidences")).withColumn("sent_evidences", sent_udf2("sent_evidences")) #withColumn("sent_evidences", explode("sent_evidences"))
df_annot2.show(n=5, truncate=False)

+--------+------------+------------+----------------------------------+-----------------------------------------+---------------+
|doc_id  |annotation  |concept_id  |concept                           |sent_evidences                           |num_occurrences|
+--------+------------+------------+----------------------------------+-----------------------------------------+---------------+
|15766526|GeneOntology|GO:0035925  |mRNA 3'-UTR AU-rich region binding|15766526.2.5,73,84                       |1              |
|15944709|ncit        |NCIT:C38784 |Apoptosis Pathway                 |15944709.2.4,102,111|15944709.2.2,103,112|2              |
|16096373|ncit        |NCIT:C18379 |E2F1 Gene                         |16096373.2.10,74,78|16096373.2.9,125,129 |2              |
|16166262|ncit        |NCIT:C134463|BCL2 Positive                     |16166262.2.3,85,100                      |1              |
|16227573|ncit        |NCIT:C80216 |Encode                            |16227573.2.1,24,30 

In [17]:
df_date = spark.read.parquet("/mnt/extproj/projekte/textmining/mx_feb24/mx_document_pubdates")
df_date.show(n=5)

+--------+----------+
|  doc_id|      date|
+--------+----------+
|22473090|2012-04-04|
|30070313|2018-08-01|
|36322282|2022-11-02|
|27121640|2016-04-28|
|37025658|2023-03-21|
+--------+----------+
only showing top 5 rows



In [18]:
df_cons2.show(n=5)

+------------+-----------+-------------+---------+-------------------+--------------------+--------------+----------+------------+
|miRNA_family|gene_family|  interaction|organisms| evidence_documents|      sent_evidences|evidence_count|sent_count|is_consensus|
+------------+-----------+-------------+---------+-------------------+--------------------+--------------+----------+------------+
|    miR-8485|       INSR| MIR_GENE;NEU|  hsa;mmu|         PMC5979287|('PMC5979287.3.54...|             1|         1|           1|
|    miR-8485|       IRS2| MIR_GENE;NEU|  hsa;mmu|         PMC5979287|('PMC5979287.3.54...|             1|         1|           1|
|    miR-8485|      ITGAM| GENE_MIR;NEU|  hsa;mmu|         PMC9639483|('PMC9639483.3.22...|             1|         1|           1|
|    miR-8485|    LAMTOR3| GENE_MIR;NEU|  hsa;mmu|         PMC9358144|('PMC9358144.3.15...|             1|         1|           0|
|    miR-8485|    LAMTOR3|MIR_GENE;DOWN|  hsa;mmu|35959437;PMC9358144|('PMC9358144.

In [19]:
df_int.show(n=5)

+------------+-----------+----------------+---------+--------------------+--------------------+--------------+----------+------------+
|miRNA_family|gene_family|     interaction|organisms|  evidence_documents|      sent_evidences|evidence_count|sent_count|is_consensus|
+------------+-----------+----------------+---------+--------------------+--------------------+--------------+----------+------------+
|    miR-8485|       INSR| {MIR_GENE, NEU}|  hsa;mmu|        [PMC5979287]|[{PMC5979287.3.54...|             1|         1|           1|
|    miR-8485|       IRS2| {MIR_GENE, NEU}|  hsa;mmu|        [PMC5979287]|[{PMC5979287.3.54...|             1|         1|           1|
|    miR-8485|      ITGAM| {GENE_MIR, NEU}|  hsa;mmu|        [PMC9639483]|[{PMC9639483.3.22...|             1|         1|           1|
|    miR-8485|    LAMTOR3| {GENE_MIR, NEU}|  hsa;mmu|        [PMC9358144]|[{PMC9358144.3.15...|             1|         1|           0|
|    miR-8485|    LAMTOR3|{MIR_GENE, DOWN}|  hsa;mmu|[3

In [20]:
df_annot2.show(n=5)

+--------+------------+------------+--------------------+--------------------+---------------+
|  doc_id|  annotation|  concept_id|             concept|      sent_evidences|num_occurrences|
+--------+------------+------------+--------------------+--------------------+---------------+
|15766526|GeneOntology|  GO:0035925|mRNA 3'-UTR AU-ri...|  15766526.2.5,73,84|              1|
|15944709|        ncit| NCIT:C38784|   Apoptosis Pathway|15944709.2.4,102,...|              2|
|16096373|        ncit| NCIT:C18379|           E2F1 Gene|16096373.2.10,74,...|              2|
|16166262|        ncit|NCIT:C134463|       BCL2 Positive| 16166262.2.3,85,100|              1|
|16227573|        ncit| NCIT:C80216|              Encode|  16227573.2.1,24,30|              1|
+--------+------------+------------+--------------------+--------------------+---------------+
only showing top 5 rows



In [21]:
df_sent.show(n=5)

+-----------+-----------------+--------------------+
|     doc_id|          sent_id|            sentence|
+-----------+-----------------+--------------------+
|PMC10208224|PMC10208224.3.109|(B) Penalty plot ...|
|PMC10208224|PMC10208224.3.110|(C) Distribution ...|
|PMC10208224|PMC10208224.3.111|(D) The error var...|
|PMC10208224|PMC10208224.3.112|(E) Identificatio...|
|PMC10208224|PMC10208224.3.113|(F) 7 immune cell...|
+-----------+-----------------+--------------------+
only showing top 5 rows



In [22]:
df_cons2.toPandas().to_sql(name='mx', con=con, if_exists='append', index_label="index")

In [23]:
df_intdoc.toPandas().to_sql(name='mx_int', con=con, if_exists='append', index_label="index")

In [24]:
df_annot2.toPandas().to_sql(name='mx_annot', con=con, if_exists='append', index_label="index")

In [25]:
df_sent.toPandas().to_sql(name='mx_sent', con=con, if_exists='append', index_label="index")

In [26]:
df_date.toPandas().to_sql(name='mx_dates', con=con, if_exists='append', index_label="index")

In [27]:
a = cur.execute("SELECT * FROM mx ORDER BY evidence_count desc LIMIT 10")
for x in a:
    print(x)

(33004, 'miR-146a', 'TRAF6', 'MIR_GENE;NEU', 'hsa;mmu', 'PMC4355369;PMC4728582;PMC3864784;PMC4782155;PMC3296165;19596990;PMC8065171;PMC5209629;PMC4118904;PMC6894150;27889748;PMC6108336;PMC3896800;27904767;PMC5477577;PMC3977468;35836229;PMC4745677;PMC4690884;24962817;PMC9618628;PMC10470528;PMC7364237;PMC7012207;24809256;PMC5514914;PMC6132771;34601384;28720071;PMC6869321;PMC7564019;24022569;PMC10186648;PMC9036177;PMC3821651;PMC6527808;PMC4632746;PMC8005601;PMC5854688;PMC6026810;PMC4425084;PMC3728531;PMC3438039;PMC4271720;PMC5358779;PMC4881486;PMC6531989;PMC3521996;PMC9160310;PMC4087671;PMC9981676;PMC7598604;PMC9022539;PMC5356798;PMC9637324;PMC9359068;PMC10734085;38025715;29048658;PMC10284807;PMC4050406;PMC7177468;26458771;PMC4033429;33002743;31330207;PMC9319398;PMC6901988;25515214;PMC5297674;PMC7522358;PMC3827083;PMC9265316;PMC9972678;PMC5760497;PMC7222555;PMC10526236;PMC10669845;PMC9413930;PMC5406027;PMC5498339;PMC5703983;PMC7075948;29754557;PMC8084815;PMC2818025;PMC3342163;PMC3938074;2

In [28]:
con.close()

In [3]:
con =  sqlite3.connect(dbFile)
cur = con.cursor()

In [4]:
a = cur.execute("SELECT * FROM mx where gene_family='MYC' and mirna_family='let-7a'")
for x in a:
    print(x)

(155177, 'let-7a', 'MYC', 'MIR_GENE;DOWN', 'hsa;mmu', 'PMC2133370;23676502;PMC5862327;PMC8155106;PMC3563491;PMC10130400;PMC6805781;PMC5683570;PMC4578885;30531691;PMC9179868;PMC7449869;PMC4536995;PMC5302913;PMC5302924;17942906;PMC10228081;PMC9562880;PMC5685744;PMC7369991;28494471;PMC9440497;PMC3382602;PMC4188135;PMC5269535;PMC3634476;PMC4742387;PMC5500619;PMC5209603;PMC5607853;21903590;20033209;PMC6796902;PMC3874662;PMC4385600;PMC10455604;PMC5979478;25077964;PMC10295928;PMC5005617;PMC10537407;PMC8417529;PMC4547013;PMC5289991;PMC7016591', "('PMC10295928.3.35', (13, 19), (97, 102));('PMC10228081.3.319', (116, 122), (126, 129));('PMC5685744.3.144', (82, 88), (109, 112));('PMC10228081.3.159', (153, 159), (174, 177));('PMC9179868.3.107', (156, 163), (51, 56));('PMC5685744.3.119', (143, 150), (78, 81));('17942906.2.11', (70, 76), (19, 22));('PMC7449869.3.43', (101, 107), (47, 50));('PMC5685744.3.146', (57, 63), (128, 131));('PMC10537407.3.126', (113, 121), (14, 17));('PMC5500619.3.50', (6, 12

In [5]:
#Total Document with miRNA-gene interaction

a = cur.execute("SELECT count( distinct doc_id) FROM mx_sent")
for x in a:
    print(x)

(102611,)


In [6]:
# Number of different genes
a = cur.execute("SELECT count( distinct gene_family) FROM mx")
for x in a:
    print(x)
    
#Number of different miRNAs (precursor level)
a = cur.execute("SELECT count( distinct mirna_family) FROM mx")
for x in a:
    print(x)

(14113,)
(2018,)


In [7]:
a = cur.execute("SELECT interaction, count(interaction) FROM mx WHERE is_consensus = 1 GROUP BY interaction;")
for x in a:
    print(x)

('GENE_MIR;DOWN', 9320)
('GENE_MIR;NEU', 43767)
('GENE_MIR;UP', 10464)
('MIR_GENE;DOWN', 53161)
('MIR_GENE;NEU', 117698)
('MIR_GENE;UP', 25294)


In [8]:
a = cur.execute("SELECT count(distinct concat_ws(' ', gene_family, mirna_family)) FROM mx WHERE is_consensus = 1;")
for x in a:
    print(x)

(196438,)


In [9]:
def get_relation_data(cursor, mirna, gene, interaction):
    
    interaction_condition = ""
    if not interaction is None:
        interaction_condition = "and interaction = '{}'".format(interaction)
    
    if mirna and gene:
        sqlres = cursor.execute("""
                    SELECT miRNA_family, gene_family, interaction, evidence_documents FROM mx WHERE miRNA_family='{}' and gene_family='{}' {}
                    ORDER BY evidence_count desc;
                    """.format(mirna, gene, interaction_condition)
                    )
    else:
        if mirna:
            query_equal = "miRNA_family='{}'".format(mirna)
        else:
            query_equal = "gene_family='{}'".format(gene)
        
        sqlres = cursor.execute("""
                SELECT miRNA_family, gene_family, interaction, evidence_documents FROM mx WHERE {} {}
                ORDER BY evidence_count desc;
                """.format(query_equal, interaction_condition)
                )
    
    results = []
    for x in sqlres:
        results.append({
            "miRNA_family": x[0],
            "gene_family": x[1],
            "interaction": x[2],
            "evidence_documents": x[3],
        })
    return results

In [10]:
get_relation_data(cur, "miR-124", "CCL2", "MIR_GENE;NEU")

[{'miRNA_family': 'miR-124',
  'gene_family': 'CCL2',
  'interaction': 'MIR_GENE;NEU',
  'evidence_documents': 'PMC8201518'}]

In [11]:
def get_annot_data(cursor, documents, rel_annotations=['celllines', 'disease', 'GeneOntology']):
    sqlres = cursor.execute("""
                SELECT doc_id, annotation, concept_id, concept FROM mx_annot WHERE doc_id IN ({}) and annotation IN ({});
                """.format(",".join(["'{}'".format(x) for x in documents]), ",".join(["'{}'".format(x) for x in rel_annotations]))
                )
    
    results = []
    for x in sqlres:
        results.append({
            "doc_id": x[0],
            "annotation": x[1],
            "concept_id": x[2],
            "concept": x[3],
        })
    return results

In [15]:
annotCounter = Counter([x["concept"] for x in get_annot_data(cur, ['PMC8201518'])])
print(annotCounter.most_common())

[('IgE immunoglobulin complex, circulating', 1), ('sarcoidosis', 1), ('melanoma', 1), ('mesenchymal stem cell', 1), ('primary cutaneous amyloidosis', 1), ('B cell receptor complex', 1), ('bacterial microcompartment', 1), ('immunoglobulin complex, circulating', 1), ('dermatitis', 1), ('MAP kinase activity', 1), ('MAPK cascade', 1), ('mast cell activation', 1), ('lung macrophage', 1), ('obsolete focal adhesion kinase activity', 1), ('lung carcinoma', 1), ('immunoglobulin complex', 1), ('eosinophil', 1), ('cell population proliferation', 1), ('leukemia', 1), ('reactive oxygen species biosynthetic process', 1), ('mast cell', 1), ('cancer', 1), ('carcinoma', 1), ('autophagy', 1), ('developmental process', 1), ('pore complex', 1), ('allergic asthma', 1), ('nascent polypeptide-associated complex', 1), ('angiogenesis', 1), ('miRNA processing', 1), ('biosynthetic process', 1), ('B cell', 1), ('threonine-type endopeptidase activity', 1), ('extracellular exosome', 1), ('asthma', 1), ('hexosaminid

In [16]:
len(annotCounter)

51

In [17]:
get_annot_data(cur, ['PMC8201518'])

[{'doc_id': 'PMC8201518',
  'annotation': 'GeneOntology',
  'concept_id': 'GO:0071743',
  'concept': 'IgE immunoglobulin complex, circulating'},
 {'doc_id': 'PMC8201518',
  'annotation': 'disease',
  'concept_id': 'DOID:11335',
  'concept': 'sarcoidosis'},
 {'doc_id': 'PMC8201518',
  'annotation': 'disease',
  'concept_id': 'DOID:1909',
  'concept': 'melanoma'},
 {'doc_id': 'PMC8201518',
  'annotation': 'celllines',
  'concept_id': 'CL:0000134',
  'concept': 'mesenchymal stem cell'},
 {'doc_id': 'PMC8201518',
  'annotation': 'disease',
  'concept_id': 'DOID:0050639',
  'concept': 'primary cutaneous amyloidosis'},
 {'doc_id': 'PMC8201518',
  'annotation': 'GeneOntology',
  'concept_id': 'GO:0019815',
  'concept': 'B cell receptor complex'},
 {'doc_id': 'PMC8201518',
  'annotation': 'GeneOntology',
  'concept_id': 'GO:0031469',
  'concept': 'bacterial microcompartment'},
 {'doc_id': 'PMC8201518',
  'annotation': 'GeneOntology',
  'concept_id': 'GO:0042571',
  'concept': 'immunoglobulin c

In [19]:

def sqlite_table_schema(conn, name):
    
    cursor = conn.execute("SELECT * FROM {} LIMIT 1;".format(name))
    sql = cursor.fetchone()
    print(sql)
    cursor.close()
    
    
    cursor = conn.execute("SELECT sql FROM sqlite_master WHERE name=?;", [name])
    sql = cursor.fetchone()[0]
    cursor.close()
    return sql



In [20]:
for table in ["mx", "mx_int", "mx_annot", "mx_sent", "mx_dates"]:
    print(sqlite_table_schema(con, table))
    print()

(0, 'miR-8485', 'INSR', 'MIR_GENE;NEU', 'hsa;mmu', 'PMC5979287', "('PMC5979287.3.54', (149, 157), (275, 291))", 1, 1, 1)
CREATE TABLE "mx" (
"index" INTEGER PRIMARY KEY AUTOINCREMENT,
"miRNA_family" TEXT,
"gene_family" TEXT,
"interaction" TEXT,
"organisms" TEXT,
"evidence_documents" TEXT,
"sent_evidences" TEXT,
"evidence_count" INTEGER,
"sent_count" INTEGER,
"is_consensus" INTEGER
)

(0, 'miR-8485', 'INSR', 'MIR_GENE;NEU', 'PMC5979287')
CREATE TABLE "mx_int" (
"index" INTEGER PRIMARY KEY AUTOINCREMENT,
"miRNA_family" TEXT,
"gene_family" TEXT,
"interaction" TEXT,
"evidence_documents" TEXT
)

(0, '15766526', 'GeneOntology', 'GO:0035925', "mRNA 3'-UTR AU-rich region binding", 1, '15766526.2.5,73,84')
CREATE TABLE "mx_annot" (
"index" INTEGER PRIMARY KEY AUTOINCREMENT,
"doc_id" TEXT,
"annotation" TEXT,
"concept_id" TEXT,
"concept" TEXT,
"num_occurrences" INTEGER,
"sent_evidences" TEXT
)

(0, 'PMC10208224', 'PMC10208224.3.109', '(B) Penalty plot of 22 immune cells in the LASSO model, error 

In [21]:
import pandas as pd
db_df = pd.read_sql_query("SELECT * FROM mx", con)
db_df.to_csv('mxplore_table_mx.tsv', index=False, sep="\t")